<a href="https://colab.research.google.com/github/yerna2772/Trial_task_CSsl/blob/main/Text_Improvement_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.5 MB/s eta 0:00:00


In [ ]:
import re
import numpy as np
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import sys
from pprint import pprint
import pandas as pd

file_path_text = '/content/sample_text.txt'
with open(file_path_text, 'r') as file:
    Text = file.read()

standardized_phrases_df = pd.read_csv('/content/Standardised terms.csv')
column_name = 'Optimal performance'
standardized_phrases = [column_name] + standardized_phrases_df[column_name].tolist()



model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)



sentences = re.split(r'[.!?]\s', Text)

similarity_threshold = 0.6

def generate_ngrams(text, n):
  words = text.split()
  ngrams = [' '.join(words[i:i + n]) for i in range(len(words) - n + 1)]
  return ngrams

for sentence in sentences:
  phrases = generate_ngrams(sentence, 3)
  max_similarity_score = 0
  for phrase in phrases :

    input_tokens = tokenizer(phrase, padding=True, truncation=True, return_tensors="pt")

    input_embeddings = model(**input_tokens).last_hidden_state.mean(dim=1)

    for standard_phrase in standardized_phrases:
      standard_tokens = tokenizer(standard_phrase, padding=True, truncation=True, return_tensors="pt")
      standard_embeddings = model(**standard_tokens).last_hidden_state.mean(dim=1)

      similarity_scores = cosine_similarity(input_embeddings.detach().numpy(), standard_embeddings.detach().numpy())

      suggestions = []

      if similarity_scores[0][0] >= similarity_threshold and similarity_scores[0][0] > max_similarity_score:
        max_similarity_score = similarity_scores[0][0]
        best_sentence = sentence
        best_phrase = phrase
        best_standard_phrase = standard_phrase

  if max_similarity_score >= similarity_threshold:
    pprint(f"Best sentence: {best_sentence}, Best phrase: {best_phrase}, Best standart phase {best_standard_phrase}, Score: {max_similarity_score}")

  else:
    pprint("No suitable recommendation found.")

("Best sentence: In today's meeting, we discussed a variety of issues "
 'affecting our department, Best phrase: variety of issues, Best standart '
 'phase Foster innovation, Score: 0.728350043296814')
('Best sentence: The weather was unusually sunny, a pleasant backdrop to our '
 'serious discussions, Best phrase: a pleasant backdrop, Best standart phase '
 'Gauge effectiveness, Score: 0.6793681383132935')
